<a href="https://colab.research.google.com/github/mohsenperfection/ML/blob/main/Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center">Introduction to Machine Learning - 25737-2</h1>
<h4 align="center">Dr. R. Amiri</h4>
<h4 align="center">Sharif University of Technology, Spring 2024</h4>


**<font color='red'>Plagiarism is strongly prohibited!</font>**


**Student Name**: **Mohsen Kamalabadi Farahani**

**Student ID**: **Mohsen Kamalabadi Farahani**





## Importing Libraries

First we import libraries that we need for this assignment.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import any other libraries needed below

## Reading Data and Preprocessing

In this section, we want to read data from a CSV file and then preprocess it to make it ready for the rest of the problem.

First, we read the data in the cell below and extract an $m \times n$ matrix, $X$, and an $m \times 1$ vector, $Y$, from it, which represent our knowledge about the features of the data (`X1`, `X2`, `X3`) and the class (`Y`), respectively. Note that by $m$, we mean the number of data points and by $n$, we mean the number of features.

In [ ]:
X, Y = None, None

### START CODE HERE ###

# Load the datase
data = pd.read_csv('data_logistic.csv')

# Feature columns
X = data[['X1', 'X2', 'X3']].values
Y = data['Y'].values.reshape(-1, 1)  # Reshaping to make Y a column vector

### END CODE HERE ###

print(X.shape)
print(Y.shape)

(10000, 3)
(10000, 1)


Next, we should normalize our data. For normalizing a vector $\mathbf{x}$, a very common method is to use this formula:

$$
\mathbf{x}_{norm} = \dfrac{\mathbf{x} - \overline{\mathbf{x}}}{\sigma_\mathbf{x}}
$$

Here, $\overline{x}$ and $\sigma_\mathbf{x}$ denote the mean and standard deviation of vector $\mathbf{x}$, respectively. Use this formula and store the new $X$ and $Y$ vectors in the cell below.

**Question**: Briefly explain why we need to normalize our data before starting the training.

**Answer**:

Normalization is a crucial step in data preprocessing for several reasons:

1. **Scale Invariance**: Many machine learning algorithms are sensitive to the scale of the features. Normalizing the data ensures that each feature contributes proportionally to the result, regardless of its original scale. This helps prevent certain features from dominating the model simply because they have larger magnitudes.

2. **Convergence Speed**: Normalizing the data often helps optimization algorithms converge more quickly. By scaling the features to a similar range, gradient descent methods can navigate the loss landscape more efficiently.

3. **Regularization**: Some regularization techniques, like L1 and L2 regularization, implicitly assume that all features are on the same scale. Normalization helps maintain this assumption, improving the effectiveness of regularization.

4. **Interpretability**: Normalized data can be easier to interpret. When features are on different scales, it becomes challenging to compare their coefficients' magnitudes and infer their relative importance in the model.

In [ ]:
### START CODE HERE ###

# Normalize X & Y
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std
Y_mean = Y.mean()
Y_std = Y.std()
Y = (Y - Y_mean) / Y_std

### END CODE HERE ###

print(X[:5])  # Print the first 5 rows to see some of the normalized data
print(Y[:5])  # Print the first 5 rows of Y


[[-1.00066149 -0.60536985 -0.85021999]
 [-1.45366949  1.73051062 -0.20238503]
 [ 0.26239007  1.69140966  0.64234794]
 [-0.68111085 -1.28703215 -0.70744413]
 [ 0.4033742   0.47210428 -0.36768115]]
[[ 0.70832689]
 [ 0.70832689]
 [ 0.70832689]
 [-1.41177754]
 [ 0.70832689]]


Finally, we should add a column of $1$s at the beginning of $X$ to represent the bias term. Do this in the next cell. Note that after this process, $X$ should be an $m \times (n+1)$ matrix.

In [ ]:
### START CODE HERE ###

# Adding a column
X = np.hstack((np.ones((X.shape[0], 1)), X))

### END CODE HERE ###

print(X.shape)  # This should print (m, n+1)
print(X[:5])    # Print the first 5 rows to see some of the data with the bias term


(10000, 5)
[[ 1.          1.         -1.00066149 -0.60536985 -0.85021999]
 [ 1.          1.         -1.45366949  1.73051062 -0.20238503]
 [ 1.          1.          0.26239007  1.69140966  0.64234794]
 [ 1.          1.         -0.68111085 -1.28703215 -0.70744413]
 [ 1.          1.          0.4033742   0.47210428 -0.36768115]]


## Training Model

### Sigmoid Function
You should begin by implementing the $\sigma(\mathbf{x})$ function. Recall that the logistic regression hypothesis $\mathcal{h}()$ is defined as:
$$
\mathcal{h}_{\theta}(\mathbf{x}) = \mathcal{g}(\theta^\mathbf{T}\mathbf{x})
$$
where $\mathcal{g}()$ is the sigmoid function as:
$$
\mathcal{g}(\mathbf{z}) = \frac{1}{1+exp^{-\mathbf{z}}}
$$
The Sigmoid function has the property that $\mathbf{g}(+\infty)\approx 1$ and $\mathcal{g}(−\infty)\approx0$. Test your function by calling `sigmoid(z)` on different test samples. Be certain that your sigmoid function works with both vectors and matrices - for either a vector or a matrix, your function should perform the sigmoid function on every element.

In [ ]:
def sigmoid(Z):
    '''
    Applies the sigmoid function to each element of Z.
    Parameters:
        Z (numpy array): Can be a scalar, vector, or matrix.
    Returns:
        numpy array: Sigmoid applied elementwise, same shape as Z.
    '''
    return 1 / (1 + np.exp(-Z))

# Test the sigmoid function
test_values = np.array([-10, -1, 0, 1, 10])
print("Sigmoid of test values:", sigmoid(test_values))

# Test with a matrix
test_matrix = np.array([[-1, 0, 1], [10, -10, 0]])
print("Sigmoid of test matrix:\n", sigmoid(test_matrix))


Sigmoid of test values: [4.53978687e-05 2.68941421e-01 5.00000000e-01 7.31058579e-01
 9.99954602e-01]
Sigmoid of test matrix:
 [[2.68941421e-01 5.00000000e-01 7.31058579e-01]
 [9.99954602e-01 4.53978687e-05 5.00000000e-01]]


### Cost Function
Implement the functions to compute the cost function. Recall the cost function for logistic regression is a scalar value given by:
$$
\mathcal{J}(\theta) = \sum_{i=1}^{n}[-y^{(i)}\log{(\mathcal{h}_\theta(\mathbf{x}^{(i)}))}-(1-y^{(i)})\log{(1-\mathcal{h}_\theta(\mathbf{x}^{(i)}))}] + \frac{\lambda}{2}||\theta||_2^2
$$

In [ ]:
import numpy as np

def computeCost(theta, X, y, regLambda):
    '''
    Computes the logistic regression cost with regularization.
    Arguments:
        theta (numpy array): A d-dimensional numpy vector.
        X (numpy array): An n-by-d numpy matrix of features.
        y (numpy array): An n-dimensional numpy vector of target values.
        regLambda (float): The scalar regularization constant.
    Returns:
        float: The computed scalar value of the cost.
    '''
    m = y.size  # number of training examples
    h = sigmoid(X @ theta)  # hypothesis function, matrix-vector product

    # Cross-entropy loss
    term1 = np.dot(y.T, np.log(h))
    term2 = np.dot((1 - y).T, np.log(1 - h))
    loss = -(term1 + term2) / m

    # Regularization (excluding the bias term theta[0])
    reg = regLambda / (2 * m) * np.dot(theta[1:], theta[1:])

    # Total cost
    total_cost = loss + reg
    return total_cost.item()  # Ensure it's a scalar, not a numpy array with one element

# Example usage:
# Assume some theta, X, y, and regLambda values
theta = np.array([0.1, 0.2, 0.3])
X = np.array([[1, 2], [1, 3], [1, 4]])
y = np.array([0, 1, 0])
regLambda = 0.01

print(computeCost(theta, X, y, regLambda))  # Should print the calculated cost


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 2)

### Gradient of the Cost Function
Now, we want to calculate the gradient of the cost function. The gradient of the cost function is a d-dimensional vector.\
We must be careful not to regularize the $\theta_0$ parameter (corresponding to the first feature we add to each instance), and so the 0's element is given by:
$$
\frac{\partial \mathcal{J}(\theta)}{\partial \theta_0} = \sum_{i=1}^n (\mathcal{h}_\theta(\mathbf{x}^{(i)})-y^{(i)})
$$

Question: What is the answer to this problem for the $j^{th}$ element (for $j=1...d$)?

Answer:

In [ ]:
import numpy as np

def computeGradient(theta, X, y, regLambda):
    '''
    Computes the gradient of the objective function
    Arguments:
        theta is d-dimensional numpy vector
        X is a n-by-d numpy matrix
        y is an n-dimensional numpy vector
        regLambda is the scalar regularization constant
    Returns:
        the gradient, a d-dimensional vector
    '''
    m, n = X.shape  # m is the number of training examples, n is the number of features
    h = sigmoid(X.dot(theta))  # Compute the hypothesis for all examples
    error = h - y  # Error term

    # Gradient calculation
    grad = (1 / m) * (X.T.dot(error))  # Base gradient without regularization

    # Regularization term for all but the bias term (first element of theta)
    reg_term = (regLambda / m) * np.copy(theta)
    reg_term[0] = 0  # No regularization for the bias term

    # Combine the base gradient with the regularization term
    grad += reg_term

    return grad

def sigmoid(z):
    '''
    Sigmoid function for logistic regression
    '''
    return 1 / (1 + np.exp(-z))



### Training and Prediction
Once you have the cost and gradient functions complete, implemen tthe fit and predict methods.\
Your fit method should train the model via gradient descent, relying on the cost and gradient functions. This function should return two parameters. The first parameter is $\theta$, and the second parameter is a `numpy` array that contains the loss in each iteration. This array is indicated by `loss_history` in the code.\
Instead of simply running gradient descent for a specific number of iterations, we will use a more sophisticated method: we will stop it after the solution hasconverged. Stop the gradient descent procedure when $\theta$ stops changing between consecutive iterations. You can detect this convergence when:
$$
||\theta_{new}-\theta_{old}||_2 <= \epsilon,
$$
for some small $\epsilon$ (e.g, $\epsilon=10E-4$).\
For readability, we’d recommend implementing this convergence test as a dedicated function `hasConverged`.

In [ ]:
def fit(X, y, regLambda=0.01, alpha=0.01, epsilon=1e-4, maxNumIters=100):
    '''
    Trains the model using gradient descent
    Arguments:
        X           : n-by-d numpy matrix
        y           : n-dimensional numpy vector
        regLambda   : scalar regularization constant
        alpha       : gradient descent learning rate
        epsilon     : convergence threshold
        maxNumIters : maximum number of gradient descent iterations
    Returns:
        theta       : learned parameters
        loss_history: list of loss values per iteration
    '''
    m, n = X.shape  # Number of examples and number of features
    theta = np.zeros(n)  # Initialize parameters
    loss_history = []  # To store the cost at each iteration

    for _ in range(maxNumIters):
        # Compute gradient and cost
        grad = computeGradient(theta, X, y, regLambda)
        cost = computeCost(theta, X, y, regLambda)
        loss_history.append(cost)

        # Update parameters
        theta_new = theta - alpha * grad

        # Check for convergence
        if hasConverged(theta, theta_new, epsilon):
            theta = theta_new
            break

        theta = theta_new

    return theta, loss_history





def hasConverged(theta_old, theta_new, epsilon):
    '''
    Checks if the optimization has converged
    Arguments:
        theta_old : previous parameter vector
        theta_new : current parameter vector
        epsilon   : convergence threshold
    Returns:
        Boolean indicating whether convergence criterion has been met
    '''
    # Calculate the Euclidean norm of the difference between old and new theta
    diff = np.linalg.norm(theta_new - theta_old)
    return diff <= epsilon


Finally, we want to evaluate our loss for this problem. Complete the cell below to calculate and print the loss of each iteration and the final theta of your model.

In [ ]:
# Assuming X and Y are already defined and preprocessed appropriately
theta, loss_history = fit(X, Y)  # calculating theta and loss of each iteration

### START CODE HERE ###

# Print the final theta
print("Final theta parameters:", theta)

# Plot the loss over iterations
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(loss_history, label='Loss per iteration')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Loss History during Gradient Descent')
plt.legend()
plt.grid(True)
plt.show()

### END CODE HERE ###


ValueError: operands could not be broadcast together with shapes (5,10000) (5,) (5,10000) 

### Testing Your Implementation
To test your logistic regression implementation, first you should use `train_test_split` function to split dataset into three parts:

- 70% for the training set
- 20% for the validation set
- 10% for the test set

Do this in the cell below.

In [ ]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = None, None, None, None, None, None

### START CODE HERE ###

### END CODE HERE ###

Then, you should complete `predict` function to find the weight vector and the loss on the test data.

In [ ]:
def predict(X, theta):
    '''
    Use the model to predict values for each instance in X
    Arguments:
        theta is d-dimensional numpy vector
        X     is a n-by-d numpy matrix
    Returns:
        an n-dimensional numpy vector of the predictions, the output should be binary (use h_theta > .5)
    '''

    Y = None
    ### START CODE HERE ###

    ### END CODE HERE ###
    return Y

Now, run the `fit` and `predict` function for different values of the learning rate and regularization constant. Plot the `loss_history` of these different values for train and test data both in the same figure.

**Question**: Discuss the effect of the learning rate and regularization constant and find the best values of these parameters.

**Answer**:

In [ ]:
### START CODE HERE ###

### END CODE HERE ###

## Naive Bayes

In this part, you will use the `GaussianNB` classifier to classify the data. You should not change the default parameters of this classifier. First, train the classifier on the training set and then find the accuracy of it on the test set.

**Question**: What is the accuracy of this method on test set?

**Answer**:

**Explanation of the Code:**

- **Data Loading and Preparation:** The dataset is loaded using pandas, and feature columns (`X1`, `X2`, `X3`) are extracted along with the target column (`Y`). Ensure that the path and column names match your dataset.
- **Training/Test Split:** The dataset is randomly split into a training set (70%) and a test set (30%).
- **GaussianNB Classifier:** A `GaussianNB` classifier is instantiated with default parameters, trained on the training set, and used to make predictions on the test set.
- **Accuracy Calculation:** The `accuracy_score` function computes the accuracy of the classifier by comparing the predicted labels to the actual labels in the test set.

Replace `"path_to_your_data.csv"` with the actual path to your dataset. Adjust the feature and label column names according to your dataset's schema. This script will print the accuracy of the GaussianNB classifier on your test data, providing a direct answer to how well the classifier performs on your dataset.


In [ ]:
### START CODE HERE ###
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Load your data
data = pd.read_csv('Q3/data_logistic.csv')
X = data[['X1', 'X2', 'X3']].values  # Adjust column names as necessary
y = data['Y'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create GaussianNB instance
gnb = GaussianNB()

# Train the model
gnb.fit(X_train, y_train)

# Predict on the test set
y_pred = gnb.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of GaussianNB on the test set: {:.2f}%".format(accuracy * 100))

### END CODE HERE ###

Accuracy of GaussianNB on the test set: 93.97%


## LDA (Linear Discriminant Analysis)

In this part, you will use the `LinearDiscriminantAnalysis` classifier to classify the data. You should not change the default parameters of this classifier. First, train the classifier on the training set and then find the accuracy of it on the test set.

**Question**: What is the accuracy of this method on test set?

**Answer**:
**Explanation of the Code:**

1. **Data Loading:** Load your dataset using pandas. You'll need to adjust the path and the column names according to your dataset structure.

2. **Splitting the Data:** The data is split into training and testing sets using `train_test_split`. Here, 30% of the data is used for testing, which is a common split ratio.

3. **Creating and Training LDA:** An instance of `LinearDiscriminantAnalysis` is created with default parameters. The model is then trained using the training data.

4. **Making Predictions and Calculating Accuracy:** After training, predictions are made on the test set. The accuracy of these predictions is then calculated using `accuracy_score`, which compares the predicted labels with the actual labels.


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# Load your data
# Here you should load your data similarly as you did earlier or using an appropriate method for your data format
data = pd.read_csv('Q3/data_logistic.csv')
X = data[['X1', 'X2', 'X3']].values  # Adjust column names as necessary
Y = data['Y'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Create LDA instance
lda = LinearDiscriminantAnalysis()

# Train the model
lda.fit(X_train, y_train)

# Predict on the test set
y_pred = lda.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of LDA on the test set: {:.2f}%".format(accuracy * 100))


Accuracy of LDA on the test set: 99.00%


## Conclution

**Question**: What is the best method for classifying this dataset? What is the best accuracy on the test set?

**Answer**:

To determine the best method for classifying your dataset, you would compare the accuracies obtained from the different classifiers you have used. From our discussions and code examples, you likely tested at least two classifiers:

1. **Linear Discriminant Analysis (LDA)**
2. **Gaussian Naive Bayes (GaussianNB)**

The accuracy of each classifier on the test set can be obtained by running the respective code snippets provided and analyzing the results.

Here’s how you would summarize and conclude which classifier performs better:

### Step-by-Step Comparison:

1. **Evaluate Each Classifier**: Implement each classifier as discussed, ensuring the dataset is consistently split between training and testing for fair comparison. Calculate the accuracy for each classifier on the test set.

2. **Compare Accuracies**: The accuracy percentages will directly tell you which classifier has higher performance on your dataset. The classifier with the highest accuracy on the test set is generally considered the best for your specific dataset under the testing conditions used.

3. **Consider Overfitting**: Ensure that the better-performing classifier is not just memorizing the training data but generalizing well to new, unseen data (the test set). This involves looking not only at the accuracy but also potentially at other metrics like precision, recall, and the confusion matrix.

### Example Conclusion:

Suppose after running the code for both classifiers, you found:
- **LDA Accuracy**: 92%
- **GaussianNB Accuracy**: 89%

In this hypothetical case, the **LDA classifier** is the better model for this dataset as it achieves a higher accuracy on the test set. Thus, you might conclude:

"The best method for classifying this dataset is the Linear Discriminant Analysis (LDA), achieving an accuracy of 92% on the test set. This suggests that LDA is more effective at capturing the underlying patterns in this particular dataset compared to Gaussian Naive Bayes, which achieved an accuracy of 89%."

### Final Note:

When determining the "best" classifier, it's crucial to ensure the model is suitable for the operational context and computational resources available. Moreover, further tuning and validation, such as cross-validation or adjusting model parameters, could potentially improve model performance even further.